In [1]:
from queue import PriorityQueue, LifoQueue
import numpy as np
import random as rnd

In [2]:
class Cell: 
    OPEN = 0
    BLOCKED = -1
    ON_FIRE = 2
    def __init__(self, row, col, prev = None, f = 0, g = 0, h = 0):
        self.row = row
        self.col = col
        self.prev = prev
        self.f = f
        self.g = g
        self.h = h
    def __lt__(self, other):        
        if (self.row == other.row):
            return self.col > other.col
        return self.row > other.row
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col and self.prev is other.prev
    def __str__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")
    def __repr__(self):
        return "(" + str(self.row) + ", " + str(self.col) + ") :: prev=" + ("NONE" if self.prev is None else "(" + str(self.prev.row) + ", " + str(self.prev.col) + ")")

In [3]:
class Maze:
    def __init__(self, dim, p, q = 0):
        self.board =  np.zeros([dim, dim], dtype=int)
        self.dim = dim
        self.currentfire = []
        self.q = q
        self.p = p
        obstaclesList = []
        for i in range(0, dim): #range() generates a list from 0 to dim-1 which can take up a lot of memory
            for j in range(0, dim):             
                if(rnd.random() < p):
                    self.board[i, j] = Cell.BLOCKED
                    obstaclesList.append((i, j))
        self.obstacles = np.array(obstaclesList)
        self.board[0, 0] = Cell.OPEN
        self.board[dim - 1, dim - 1] = Cell.OPEN

    def isSolvable(self):
        return pathExists(self)
    
    def startFire(self):
        i = rnd.randint(0,self.dim-1)
        j = rnd.randint(0,self.dim-1)
        while(self.board[i,j] == -1 or (i==0 and j == 0) or (i == self.dim-1 and j == self.dim-1)):
            i = rnd.randint(0,self.dim-1)
            j = rnd.randint(0,self.dim-1)
        self.board[i,j] = Cell.ON_FIRE
        self.currentfire.append([i,j,None])
    
    def iterateFire(self):
        visited = [] #neighbors already visited in current iteration
        nowonfire = [] #need to append all new fires at the end 
        for f in self.currentfire:
            if(f[2] != 0): #check to see if all neighbors are already on fire. 
                neighbors = findNeighboringFlammableCoords(f[0:2],self) #calculate valid neighbors of f, the cell that is currently on fire
                ofnbor = 0 #counter to find out how many neighbors are on fire
                for n in neighbors: #loop through the neighbors of f
                    nr,nc = n 
                    if (self.board[nr,nc] == Cell.ON_FIRE): 
                        ofnbor = ofnbor + 1 
                    else:
                        if(n[0:2] not in visited): #only roll once for each cell not on fire
                            if(self.board[nr,nc] != Cell.BLOCKED):
                                surroundings = findNeighboringFlammableCoords(n[0:2],self) #see what the neighbors are for the neighbor of f
                                k = 0 #k will be guaranteed to be at least 1
                                for nn in surroundings:
                                    nnr,nnc = nn
                                    if(self.board[nnr,nnc] == Cell.ON_FIRE):
                                        k = k + 1 #counter
                                if(rnd.random() <= (1-((1-self.q)**k))): #roll
                                    self.board[nr,nc] = Cell.ON_FIRE 
                                    nowonfire.append([nr,nc,None])
                                    ofnbor = ofnbor + 1
                                else:
                                    visited.append([nr,nc])
                if(ofnbor == len(neighbors)):
                        f[2] = 0
        if(nowonfire):
            for x in nowonfire:
                self.currentfire.append(x)

In [4]:
def pathExists(maze, fromCoords = (0,0), toCoords = None):
    if toCoords is None:
        toCoords = (maze.dim - 1, maze.dim - 1)
    toRow, toCol = toCoords
    visited = set()
    fringe = LifoQueue()
    fringe.put(fromCoords)
    while not fringe.empty():
        currentCoords = fringe.get()
        currentRow, currentCol = currentCoords
        if (currentRow == toRow and currentCol == toCol):
            return True
        else:
            neighbors = findNeighboringOpenCoords(currentCoords, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                fringe.put(neighbor)
            visited.add(currentCoords)
    return False

In [5]:
def findNeighboringOpenCoords(coords, maze):
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

def findNeighboringOpenCells(cell, maze):
    neighboringCoords = findNeighboringOpenCoords((cell.row, cell.col), maze)
    neighbors = []
    for neighbor in neighboringCoords:
        row, col = neighbor
        neighbors.append(Cell(row, col, cell))
    return neighbors

def findNeighboringFlammableCoords(coords,maze): 
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] == Cell.BLOCKED):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

In [6]:
def uniformCost(cell, maze):
    return 1

def distanceBetween(fromCoords, toCoords = (0, 0)):
    fromRow, fromCol = fromCoords
    toRow, toCol = toCoords
    return abs(toRow - fromRow) + abs(toCol - fromCol)

def shortestPathSearch(maze, startCoords = (0, 0), goalCoords = None, heuristicFunction = uniformCost, findNeighbors = findNeighboringOpenCells):
    if goalCoords is None:
        goalCoords = (maze.dim - 1, maze.dim - 1)
    goalRow, goalCol = goalCoords
    startRow, startCol = startCoords
    startCell = Cell(startRow, startCol)
    visited = set()
    toBeVisited = {startCoords: (0, 0)}
    fringe = PriorityQueue()
    fringe.put((0, startCell))
    while not fringe.empty():
        fScore, currentCell = fringe.get()
        if (currentCell.row, currentCell.col) in visited:
            continue
#         toBeVisited.pop((currentCell.row, currentCell.col))
        if (currentCell.row == goalRow and currentCell.col == goalCol):
            shortestPath = []
            while (currentCell != startCell):
                shortestPath.append((currentCell.row, currentCell.col))
                currentCell = currentCell.prev
            shortestPath.append((startRow, startCol))
            shortestPath.reverse()
            return shortestPath
        else:
            neighbors = findNeighbors(cell = currentCell, maze = maze)
            for neighbor in neighbors:
                if (neighbor.row, neighbor.col) in visited:
                    continue
                distanceFromStart = distanceBetween(fromCoords = (neighbor.row, neighbor.col))
#                 if (neighbor.row, neighbor.col) in toBeVisited:
#                     prevDistanceFromStart, prevHeuristic = toBeVisited[(neighbor.row, neighbor.col)]
#                     if (distanceFromStart < prevDistanceFromStart):
#                         toBeVisited[(neighbor.row, neighbor.col)] = (prevDistanceFromStart, prevHeuristic)
#                         fringe.put((prevDistanceFromStart + prevHeuristic, neighbor))
#                     continue
                heuristic = heuristicFunction(cell = neighbor, maze = maze)
#                 toBeVisited[(neighbor.row, neighbor.col)] = (distanceFromStart, heuristic)
                newFScore = fScore + heuristic
                fringe.put((newFScore, neighbor))
            visited.add((currentCell.row, currentCell.col))          
    return None

In [10]:
# m1.startFire()
# print(m1.board)

In [9]:
# m1.iterateFire()
# print(m1.board)

In [8]:
# print(m1.currentfire)